In [1]:
import numpy as np
import pandas as pd

In [2]:
# 관광지명, 도로명주소, lat, lng, 분류
df = pd.read_excel('data/서울시 관광 문화_ljh.XLSX', engine='openpyxl')

In [3]:
df.to_csv('data/main/temp_files/서울_문화.csv', index=False)

In [4]:
df = pd.read_csv('data/main/temp_files/서울_문화.csv', encoding='utf-8', sep=',')

In [5]:
df = df[(df['언어']) == 'ko']
df.head(4)

,고유번호,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보,홈페이지 운영 언어,유모차 대여 여부
0,305,ko,세종문화회관,110-821 서울 종로구 세종로 81-3,03172 서울 종로구 세종대로 175 (세종문화회관),02-399-1000,https://www.sejongpac.or.kr/portal/main/main.do,공연에 따라 이용 시간이 다름 * 현장 및 전화 예매 AM 09:00 ~ PM 0...,월~일,NaN,"5호선 광화문역 1번, 8번 출구 도보 5분 3호선 경복궁역 6번, 7번 출구...",NaN,NaN
2,380,ko,명동실탄사격장,100-860 서울 중구 충무로2가 11-1 선샤인빌딩3층,04537 서울 중구 명동8가길 27 선샤인빌딩3층,02-777-6604,http://www.koreashootingclub.com/gnuboard4/,평일 13:00 - 20:00 주말/공휴일 11:00 - 20:00,NaN,연중무휴,"4호선 명동역 8,9번 출구",NaN,NaN
5,1466,ko,서울랜드,경기 과천시 막계동 33,"13829 경기 과천시 광명로 181 (막계동, 서울랜드)",02-509-6000,http://www.seoulland.co.kr,"09:30 ~ 18:00, 20:00, 21:00, 22:00(축제일정 및 공원사정...",월 - 일,NaN,NaN,NaN,대여 가능
6,1546,ko,예지원,100-392 서울 중구 장충동2가 201-6,04605 서울특별시 중구 장충단로 72 (장충동2가),02-2253-2211,http://www.yejiwon.or.kr,※교육별 수강시간이 상이하므로 홈페이지 참조,NaN,일요일?공휴일 휴관,3호선 동대입구 2번 출구 420번 버스 탑승 후 국립극장 앞 하차 2호선 동대문 ...,NaN,NaN


In [6]:
df.언어.unique()

array(['ko'], dtype=object)

In [7]:
# 도로명주소만 남기기
df.rename(columns= {'상호명':'관광지명', '신주소':'도로명주소'}, inplace=True)
df = df[['관광지명', '도로명주소']]
df.reset_index(drop=True,inplace=True)
df.head()

,관광지명,도로명주소
0,세종문화회관,03172 서울 종로구 세종대로 175 (세종문화회관)
1,명동실탄사격장,04537 서울 중구 명동8가길 27 선샤인빌딩3층
2,서울랜드,"13829 경기 과천시 광명로 181 (막계동, 서울랜드)"
3,예지원,04605 서울특별시 중구 장충단로 72 (장충동2가)
4,DMC 홍보관,03926 서울특별시 마포구 월드컵북로 366 (상암동)


In [8]:
df.nunique()

관광지명     200
도로명주소    198
dtype: int64

In [11]:
df.to_csv('data/서울_문화_명소.csv',index=False)

In [2]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 
import os,glob

In [2]:
df= pd.read_csv('data/서울_문화_명소.csv', encoding='utf-8', sep=',')
df.head()

,관광지명,도로명주소
0,세종문화회관,03172 서울 종로구 세종대로 175 (세종문화회관)
1,명동실탄사격장,04537 서울 중구 명동8가길 27 선샤인빌딩3층
2,서울랜드,"13829 경기 과천시 광명로 181 (막계동, 서울랜드)"
3,예지원,04605 서울특별시 중구 장충단로 72 (장충동2가)
4,DMC 홍보관,03926 서울특별시 마포구 월드컵북로 366 (상암동)


In [3]:
df.isna().sum().sum()

0

In [4]:
df['관광지명'][150]

'한강 자전거 코스'

In [5]:
df.도로명주소[150].split(' ')[0]

'143-300'

In [6]:
# 도로명주소에서 우편번호 제거
addr_list = []
for i in df.index:
    addr1 = df.도로명주소[i].split(' ')[1:]
    df.도로명주소[i]= ' '.join(addr1)
    addr_list.append(df.도로명주소[i])
    # df.도로명주소[i] = df.도로명주소[i].split(' ')[1:]
addr_list

['서울 종로구 세종대로 175  (세종문화회관) ',
 '서울 중구 명동8가길 27 선샤인빌딩3층',
 '경기 과천시 광명로 181 (막계동, 서울랜드) ',
 '서울특별시 중구 장충단로 72 (장충동2가) ',
 '서울특별시 마포구 월드컵북로 366 (상암동) ',
 '서울 영등포구 여의공원로 13 (한국방송공사) ',
 '서울 중구 명동8가길 47 (충무로2가) 동양빌딩 2층',
 '서울특별시 광진구 동일로4길 56-10 (자양동) ',
 '서울 서초구 남부순환로 2364 (서초동, 국립국악원) ',
 '서울 광진구 구의강변로 45 (구의동, 구의동 복합빌딩) 지하 2층',
 '서울 서초구 강남대로107길 5 (잠원동, 크레신사옥) ',
 '서울 강남구 학동로 342 (논현동, 에스케이 허브블루) 209',
 '서울 영등포구 여의서로 160 (여의도동) ',
 '서울 마포구 어울마당로 35 (서교동) ',
 '서울 영등포구 여의동로 330 (여의도동, 한강사업본부 여의도안내센터) ',
 '서울 구로구 경인로 430 (고척동, 고척스카이돔) ',
 ' ',
 '서울특별시 종로구 계동길 33-13 ',
 '서울 종로구 계동4길 3 (계동) ',
 '서울 종로구 북촌로5길 19-12 (재동) ',
 '서울 종로구 북촌로11길 35-16 ',
 '서울 중구 정동길 3  ',
 '서울 중구 동호로 249 (장충동2가, 신라호텔) ',
 '서울 종로구 율곡로 102 (와룡동, 서울돈화문국악당) ',
 '서울 마포구 와우산로 146 (창전동, 그레이스빌딩) ',
 '서울 성동구 연무장1길 7-1 (성수동2가) ',
 '서울 마포구 양화로 176 (동교동, 동교동 스타피카소) 8층',
 '서울 용산구 회나무로39길 33 (이태원동) ',
 '서울 종로구 옥인1길 34 (옥인동, 박노수 가옥) ',
 '서울 강남구 논현로 425 (역삼동, 마이다스빌딩) 마이다스 빌딩 2층',
 '서울 중구 소월로 50 (힐튼호텔) ',
 '서울 종로구 가회동 218 ',
 '서울 종로구

In [3]:
# 도로명주소에서 우편번호 제거
for i in df.index:
    addr1 = df.도로명주소[i].split(' ')[1:]
    df.도로명주소[i] = (' '.join(addr1))
df.도로명주소

0                      서울 종로구 세종대로 175  (세종문화회관) 
1                          서울 중구 명동8가길 27 선샤인빌딩3층
2                     경기 과천시 광명로 181 (막계동, 서울랜드) 
3                       서울특별시 중구 장충단로 72 (장충동2가) 
4                      서울특별시 마포구 월드컵북로 366 (상암동) 
                          ...                    
195           서울 동대문구 회기로 117-3 (회기동, 한국농촌경제연구원) 
196          서울 구로구 오류로 36-25 (오류동, 50플러스 남부캠퍼스) 
197               서울 강남구 봉은사로 155 (논현동, 호텔 카푸치노) 
198    서울 서초구 신반포로11길 39 (한강반포지구 내 유선장) The River
199                    서울 용산구 두텁바위로1길 69-1 (후암동) 
Name: 도로명주소, Length: 200, dtype: object

In [40]:
type(df.도로명주소.values[1])

str

In [25]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [67]:
# 2
temp1 =[]
for i in df.index:
    try:
        # target = df['관광지명'][i].strip()
        temp1.append(kakao_location(df.도로명주소.values[i]))
    except:
        print(i, df.관광지명[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

11 MBK 엔터테인먼트
16 한류관광지
56 아리랑 영화의 거리
150 한강 자전거 코스
153 서울애니메이션센터
169 CGV 씨네 라이브러리
190 호텔 더 디자이너스 삼성점


In [65]:
# 2
temp1 =[]
for i in df.index[190:]:
    try:
        # target = df['관광지명'][i].strip()
        temp1.append(kakao_location(df.도로명주소.values[i]))
    except:
        print(i, df.관광지명[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

190 호텔 더 디자이너스 삼성점


In [68]:
df2.tail(10)

,관광지명,도로명주소,lat,lng
190,호텔 더 디자이너스 삼성점,서울특별시 강남구 테헤란로79길 7 (삼성동),37.506900,127.031496
191,리얼케이팝댄스(REAL K-POP DANCE),서울시 마포구 동교로 19길 48,37.507195,126.991688
192,북촌문화센터,서울 종로구 계동길 37 (계동),37.547082,126.975475
193,남산서울타워 한복문화 체험관,"서울 용산구 남산공원길 105 (용산동2가, YTN서울타워)",NaN,NaN
194,성수동 유니온스타 볼링클럽,"서울 성동구 연무장7길 12 (성수동2가, 세대빌딩)",NaN,NaN
195,서울바이오허브,"서울 동대문구 회기로 117-3 (회기동, 한국농촌경제연구원)",NaN,NaN
196,서울시50플러스 남부캠퍼스,"서울 구로구 오류로 36-25 (오류동, 50플러스 남부캠퍼스)",NaN,NaN
197,호텔 카푸치노,"서울 강남구 봉은사로 155 (논현동, 호텔 카푸치노)",NaN,NaN
198,더리버,서울 서초구 신반포로11길 39 (한강반포지구 내 유선장) The River,NaN,NaN
199,후암서재,서울 용산구 두텁바위로1길 69-1 (후암동),NaN,NaN


In [72]:
df2.dropna()

,관광지명,도로명주소,lat,lng
0,세종문화회관,서울 종로구 세종대로 175 (세종문화회관),37.571848,126.976168
1,명동실탄사격장,서울 중구 명동8가길 27 선샤인빌딩3층,37.562001,126.986564
2,서울랜드,"경기 과천시 광명로 181 (막계동, 서울랜드)",37.436547,127.024045
3,예지원,서울특별시 중구 장충단로 72 (장충동2가),37.551638,127.002523
4,DMC 홍보관,서울특별시 마포구 월드컵북로 366 (상암동),37.577695,126.892115
...,...,...,...,...
188,씨제스사옥,"서울 강남구 도산대로 310 (논현동, 916빌딩) 5층",37.592404,127.049177
189,티켓라운지 씬디,서울 마포구 어울마당로 61 (서교동),37.488474,126.841845
190,호텔 더 디자이너스 삼성점,서울특별시 강남구 테헤란로79길 7 (삼성동),37.506900,127.031496
191,리얼케이팝댄스(REAL K-POP DANCE),서울시 마포구 동교로 19길 48,37.507195,126.991688


In [22]:
kakao_location('서울특별시마포구 월드컵북로 366 (상암동)')

(37.5776946420349, 126.89211485663)

In [ ]:
# # 검증
# temp = df2['관광지명'][10]
# rtn_addr(temp)

In [73]:
df2.shape

(200, 4)

In [74]:
# 4 처리 끝난 파일 저장
df2.to_csv('data/서울_문화_명소_final.csv',index=False)

In [3]:
# 읽어오기
df3 = pd.read_csv('data/서울_문화_명소_final.csv', encoding='utf-8', sep=',')

In [4]:
df3.head()

,관광지명,도로명주소,lat,lng
0,세종문화회관,서울 종로구 세종대로 175 (세종문화회관),37.571848,126.976168
1,명동실탄사격장,서울 중구 명동8가길 27 선샤인빌딩3층,37.562001,126.986564
2,서울랜드,"경기 과천시 광명로 181 (막계동, 서울랜드)",37.436547,127.024045
3,예지원,서울특별시 중구 장충단로 72 (장충동2가),37.551638,127.002523
4,DMC 홍보관,서울특별시 마포구 월드컵북로 366 (상암동),37.577695,126.892115


In [5]:
df3.isna().sum()

관광지명     0
도로명주소    0
lat      7
lng      7
dtype: int64

In [7]:
df3.dropna(inplace=True)
df3.isna().sum()

관광지명     0
도로명주소    0
lat      0
lng      0
dtype: int64

In [9]:
df3.tail()

,관광지명,도로명주소,lat,lng
188,씨제스사옥,"서울 강남구 도산대로 310 (논현동, 916빌딩) 5층",37.592404,127.049177
189,티켓라운지 씬디,서울 마포구 어울마당로 61 (서교동),37.488474,126.841845
190,호텔 더 디자이너스 삼성점,서울특별시 강남구 테헤란로79길 7 (삼성동),37.506900,127.031496
191,리얼케이팝댄스(REAL K-POP DANCE),서울시 마포구 동교로 19길 48,37.507195,126.991688
192,북촌문화센터,서울 종로구 계동길 37 (계동),37.547082,126.975475


In [11]:
df3.to_csv('data/서울_문화_명소_final.csv',index=False)
df4 = pd.read_csv('data/서울_문화_명소_final.csv', encoding='utf-8', sep=',')
df4.tail()

,관광지명,도로명주소,lat,lng
188,씨제스사옥,"서울 강남구 도산대로 310 (논현동, 916빌딩) 5층",37.592404,127.049177
189,티켓라운지 씬디,서울 마포구 어울마당로 61 (서교동),37.488474,126.841845
190,호텔 더 디자이너스 삼성점,서울특별시 강남구 테헤란로79길 7 (삼성동),37.506900,127.031496
191,리얼케이팝댄스(REAL K-POP DANCE),서울시 마포구 동교로 19길 48,37.507195,126.991688
192,북촌문화센터,서울 종로구 계동길 37 (계동),37.547082,126.975475
